In [ ]:
import numpy as np
import matplotlib
import time
import csv
from matplotlib import pyplot
%run lstm.ipynb

In [ ]:
# Функция вычитывает заданный столбец CSV файла.
def read_csv_column(filename, col_id):
    file = open(filename)
    reader = csv.reader(file)
    data = [float(row[col_id]) for row in reader]
    file.close()
    return data

In [ ]:
# Функция реализует разбиение данных на два набора в указанных пропорциях.
def split_data(data, ratio=0.8):
    num_samples = int(len(data) * ratio)
    return np.asfarray(data[:num_samples]), np.asfarray(data[num_samples:])

In [ ]:
# Открываем CSV файл и считываем третью колонку - там лежат нужные нам данные.
filename = 'e:\international-airline-passengers-monthly-totals-inthousands-jan-49-dec-60.csv'
data = read_csv_column(filename, 2)
train_data, test_data = split_data(data, ratio=0.8)

In [ ]:
# Нормализуем данные.
data_max = np.amax(data)
train_data /= data_max
test_data /= data_max

In [ ]:
# Рисуем на одном грпфике часть обучения и часть для тестов.
matplotlib.pyplot.plot(np.arange(0, len(train_data), 1), train_data)
matplotlib.pyplot.plot(np.arange(0, len(test_data), 1) + len(train_data), test_data)

In [ ]:
# Гиперпараметры.
num_epochs = 800
num_repeat = 1
input_dim = 12
hidden_dim = 15
learn_rate = 0.5
optimizer = 'adagrad'

In [ ]:
train_x, train_y = [], []
for i in range(len(train_data) - input_dim):
    train_x.append(train_data[i:i+input_dim])
    train_y.append(train_data[i+1:i+1+input_dim])

test_x, test_y = [], []
for i in range(len(test_data) - input_dim):
    test_x.append(test_data[i:i+input_dim])
    test_y.append(test_data[i+1:i+1+input_dim])

In [ ]:
sess = tf.Session()
net = lstm_cell(sess, input_dim, hidden_dim, learn_rate, optimizer)

begin = time.time()
for e in range(num_epochs):
    net.clear_state()
    for x, y in zip(train_x, train_y):
        loss = net.train([x], [y], repeat=num_repeat)
end = time.time()
train_time = end - begin

In [ ]:
prev_seq = []
pred_seq = []

if False:
    net.clear_state()
    prev_seq = [train_x[0]]
else:
    #net.clear_state()
    prev_seq = [train_y[-1]]
    #prev_seq = [train_x[-1]]

for i in range(len(test_data)):
    prev_seq = net.test(prev_seq)
    pred_seq.append(prev_seq[0])

In [ ]:
pred_points = []

for i in pred_seq:
    pred_points.append(i[-1])

In [ ]:
mae_error = np.sum(np.abs(pred_points - test_data))/len(pred_points)
mse_error = np.sum((pred_points - test_data)**2)/len(pred_points)

In [ ]:
matplotlib.pyplot.plot(test_data, 'orange')
#matplotlib.pyplot.plot(train_data)
matplotlib.pyplot.plot(pred_points, '--')

#matplotlib.pyplot.plot(np.arange(0, len(train_data), 1), train_data)
#matplotlib.pyplot.plot(np.arange(0, len(test_data), 1) + len(train_data), test_data)
#matplotlib.pyplot.plot(np.arange(len(train_data), len(train_data) + len(pred_curve), 1), pred_curve)

In [ ]:
# Отображаем статистику обучения.
print('optimizer', optimizer)
print('learn_rate', learn_rate)
print('train_time', train_time)
print('num_epochs', num_epochs)
print('num_repeat', num_repeat)
print('input_dim', input_dim)
print('hidden_dim', hidden_dim)
print('MAE', mae_error)
print('MSE', mse_error)